<h1 align=center><font size = 5>Opening New Restaurant in Delhi,India</font></h1>


## Introduction

Delhi is city, growing exponentially in Economical sector.With its population near about 1.9 crores it becomes one of the top market for investors.This notebook gives investors brief idea for opening new restaurants.So,let's get started.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. Build a dataframe of neighborhoods in Delhi, India by web scraping the data from Wikipedia page

2. Get the geographical coordinates of the neighborhoods

3. Obtain the venue data for the neighborhoods from Foursquare API

4. Explore and cluster the neighborhoods

5. Examining Clusters to find best probable location to open Restaurant   
</font>
</div>

## 1. Importing Necessary Packages and Libraries

In [55]:
import numpy as np 

import pandas as pd 
#pd.set_option("display.max_columns", None)
#pd.set_option("display.max_rows", None)

import json # library to handle JSON files
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


!pip install beautifulsoup4
from bs4 import BeautifulSoup # library to parse HTML and XML documents


import requests # library to handle requests



from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print("All necessary Packeges Libraries imported.")


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

All necessary Packeges Libraries imported.


## 2.Scrap data from Wikipedia page into a DataFrame

In [3]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Delhi").text

In [4]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [5]:
# create a list to store neighborhood data
neighborhoodList = []

In [6]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [7]:
# create a new DataFrame from the three lists
delhi_df = pd.DataFrame({"Neighborhood": neighborhoodList})
delhi_df.head()

,Neighborhood
0,Neighbourhoods of Delhi
1,Ashok Nagar (Delhi)
2,Ashok Vihar
3,Ashram Chowk
4,Babarpur


In [8]:
#Dropping first row of DataFrame
delhi_df=delhi_df.drop(delhi_df.index[0])

In [9]:
# get shape of the dataframe
delhi_df.shape

(135, 1)

## 3.Get the geographical coordinates

In [10]:
!pip install geocoder
import geocoder

     |████████████████████████████████| 102kB 1.9MB/s ta 0:00:01
     |████████████████████████████████| 829kB 5.9MB/s eta 0:00:01
     |████████████████████████████████| 81kB 25.6MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built future


In [15]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Delhi, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [14]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in delhi_df["Neighborhood"].tolist() ]

In [16]:
coords

[[28.692230000000052, 77.30127000000005],
 [28.690420000000074, 77.17606000000006],
 [28.710567996859677, 77.32694891535819],
 [28.507370000000037, 77.30347000000006],
 [28.507370000000037, 77.30347000000006],
 [28.652180044080556, 77.12977498073028],
 [28.800590000000057, 77.03473000000008],
 [28.549540000000036, 77.18170000000003],
 [28.69980000000004, 77.25917000000004],
 [28.595060000000046, 77.18573000000004],
 [28.656240000000025, 77.23233000000005],
 [28.538410000000056, 77.24831000000006],
 [28.676690000000065, 77.21762000000007],
 [28.633920000000046, 77.21978000000007],
 [28.607590000000073, 77.08721000000008],
 [28.654567834071344, 77.23341918654859],
 [28.62832000000003, 77.24727000000007],
 [28.605680000000064, 77.08222000000006],
 [28.560560000000066, 77.24679000000003],
 [28.573000000000036, 77.23360000000008],
 [28.591530000000034, 77.12942000000004],
 [28.594895372181515, 77.16726304796912],
 [28.684700000000078, 77.32775000000004],
 [28.679040000000043, 77.31476000000

In [17]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the coordinates into the original dataframe
delhi_df['Latitude'] = df_coords['Latitude']
delhi_df['Longitude'] = df_coords['Longitude']

# check the neighborhoods and the coordinates
print(delhi_df.shape)
delhi_df

(135, 3)


,Neighborhood,Latitude,Longitude
1,Ashok Nagar (Delhi),28.690420,77.176060
2,Ashok Vihar,28.710568,77.326949
3,Ashram Chowk,28.507370,77.303470
4,Babarpur,28.507370,77.303470
5,"Badarpur, Delhi",28.652180,77.129775
...,...,...,...
131,"Vasant Vihar, Delhi",28.646420,77.306150
132,Vivek Vihar subdivision,28.647800,77.164470
133,West Patel Nagar,28.621050,77.217100
134,Yamuna Pushta,28.700570,77.272100


In [18]:
# drop rows containing NaN values
delhi_df=delhi_df.dropna()

In [19]:
# check shape of DataFrame
delhi_df.shape

(134, 3)

In [20]:
# save the DataFrame as CSV file
delhi_df.to_csv("delhi_df.csv", index=False)

## 4.Create a map of Delhi with neighborhoods superimposed on top

In [21]:

# get the coordinates of Delhi
address = 'Delhi, India'

geolocator = Nominatim(user_agent="de_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Delhi, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Delhi, India 28.6517178, 77.2219388.


In [22]:
# create map of Delhi using latitude and longitude values
map_delhi = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(delhi_df['Latitude'], delhi_df['Longitude'], delhi_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_delhi)  
    
map_delhi

In [19]:
# save the map as HTML file
map_delhi.save('map_delhi.html')

## 5.Use the Foursquare API to explore the neighborhoods

In [23]:
CLIENT_ID = 'MERFYCISTKR5CMRMLVNOREQ302UA42ZL3S1MCPVVICJ215XS' # your Foursquare ID
CLIENT_SECRET = '4YBUSKX0KFSUV04YAOYUIW353JZUJFHF12L40KD0GJ1NECGS' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MERFYCISTKR5CMRMLVNOREQ302UA42ZL3S1MCPVVICJ215XS
CLIENT_SECRET:4YBUSKX0KFSUV04YAOYUIW353JZUJFHF12L40KD0GJ1NECGS


#### Now, let's get the top 50 venues that are within a radius of 1000 meters.

In [24]:
radius = 1000
LIMIT = 50

venues = []

for lat, long, neighborhood in zip(delhi_df['Latitude'], delhi_df['Longitude'], delhi_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [25]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2552, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Ashok Nagar (Delhi),28.69042,77.17606,Bellagio,28.696361,77.180021,Asian Restaurant
1,Ashok Nagar (Delhi),28.69042,77.17606,Domino's Pizza,28.693000,77.177000,Pizza Place
2,Ashok Nagar (Delhi),28.69042,77.17606,Subway,28.696321,77.179983,Sandwich Place
3,Ashok Nagar (Delhi),28.69042,77.17606,Subway.,28.695571,77.171964,Sandwich Place
4,Ashok Nagar (Delhi),28.69042,77.17606,Rahul Egg Corner,28.688240,77.168599,Snack Place


#### Let's check how many venues were returned for each neighorhood

In [27]:

venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Ashok Nagar (Delhi),10,10,10,10,10,10
Ashram Chowk,4,4,4,4,4,4
Babarpur,4,4,4,4,4,4
"Badarpur, Delhi",29,29,29,29,29,29
Bali Nagar,2,2,2,2,2,2
...,...,...,...,...,...,...
Vasant Kunj,44,44,44,44,44,44
"Vasant Vihar, Delhi",6,6,6,6,6,6
Vivek Vihar subdivision,13,13,13,13,13,13


#### Let's find out how many unique categories can be curated from all the returned venues

In [28]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 197 uniques categories.


In [29]:
# print out the list of categories
venues_df['VenueCategory'].unique()

array(['Asian Restaurant', 'Pizza Place', 'Sandwich Place', 'Snack Place',
       'South Indian Restaurant', 'Indian Restaurant', 'Market',
       'Dessert Shop', 'Coffee Shop', 'Train Station',
       'Light Rail Station', 'American Restaurant', 'Hookah Bar',
       'Fast Food Restaurant', 'Café', 'Multiplex', 'Shopping Mall',
       'Metro Station', 'Chinese Restaurant', 'Garden', 'Ice Cream Shop',
       'Lounge', 'Gym', 'Playground', 'Mobile Phone Shop',
       'Airport Food Court', 'Hotel', 'History Museum', 'Clothing Store',
       'Grocery Store', 'Bank', 'Tourist Information Center',
       'IT Services', 'Botanical Garden', 'Business Service', 'Park',
       'Karnataka Restaurant', 'Moroccan Restaurant', 'Pub', 'Nightclub',
       'Museum', 'Mosque', 'Boutique', 'Monument / Landmark',
       'Food & Drink Shop', 'Paper / Office Supplies Store',
       'Hardware Store', 'Flea Market', 'Pharmacy', 'Historic Site',
       'Frozen Yogurt Shop', 'Restaurant', 'Tibetan Restaurant',


## 6. Analyze Each Neighborhood

In [30]:
# one hot encoding
delhi_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
delhi_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [delhi_onehot.columns[-1]] + list(delhi_onehot.columns[:-1])
delhi_onehot = delhi_onehot[fixed_columns]

print(delhi_onehot.shape)
delhi_onehot.head()

(2552, 198)


,Neighborhoods,ATM,Afghan Restaurant,Airport,Airport Food Court,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,...,Tourist Information Center,Trail,Train Station,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Women's Store,Yoga Studio
0,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
# Let us see name of each VenueCategory
delhi_onehot.columns.tolist()

['Neighborhoods',
 'ATM',
 'Afghan Restaurant',
 'Airport',
 'Airport Food Court',
 'Airport Terminal',
 'American Restaurant',
 'Antique Shop',
 'Arcade',
 'Art Gallery',
 'Art Museum',
 'Asian Restaurant',
 'Athletics & Sports',
 'Australian Restaurant',
 'BBQ Joint',
 'Bagel Shop',
 'Bakery',
 'Bank',
 'Bar',
 'Basketball Court',
 'Bed & Breakfast',
 'Beer Garden',
 'Bengali Restaurant',
 'Big Box Store',
 'Bike Shop',
 'Bistro',
 'Bookstore',
 'Botanical Garden',
 'Boutique',
 'Breakfast Spot',
 'Bridal Shop',
 'Buffet',
 'Burger Joint',
 'Burrito Place',
 'Bus Station',
 'Business Service',
 'Café',
 'Cheese Shop',
 'Chinese Restaurant',
 'Chocolate Shop',
 'Clothing Store',
 'Cocktail Bar',
 'Coffee Shop',
 'College Cafeteria',
 'Comfort Food Restaurant',
 'Construction & Landscaping',
 'Convenience Store',
 'Cosmetics Shop',
 'Deli / Bodega',
 'Department Store',
 'Design Studio',
 'Dessert Shop',
 'Dim Sum Restaurant',
 'Diner',
 'Donut Shop',
 'Dumpling Restaurant',
 'Eastern 

#### Let us filter data where only 'Restaurant' as VenueCategory is present

In [34]:
# filter neighborhood names that contain the word Restaurants
Reastaurant_names = list(venues_df.VenueCategory.unique())

VenueCategory_with_Restaurants = []

for x in Reastaurant_names:
    if "Restaurant" in x:
        VenueCategory_with_Restaurants.append(x)
        
VenueCategory_with_Restaurants

['Asian Restaurant',
 'South Indian Restaurant',
 'Indian Restaurant',
 'American Restaurant',
 'Fast Food Restaurant',
 'Chinese Restaurant',
 'Karnataka Restaurant',
 'Moroccan Restaurant',
 'Restaurant',
 'Tibetan Restaurant',
 'Italian Restaurant',
 'Thai Restaurant',
 'Mediterranean Restaurant',
 'French Restaurant',
 'Molecular Gastronomy Restaurant',
 'North Indian Restaurant',
 'Portuguese Restaurant',
 'Japanese Restaurant',
 'Vegetarian / Vegan Restaurant',
 'Afghan Restaurant',
 'Vietnamese Restaurant',
 'Dumpling Restaurant',
 'Multicuisine Indian Restaurant',
 'Mexican Restaurant',
 'Dim Sum Restaurant',
 'Korean Restaurant',
 'Australian Restaurant',
 'Tapas Restaurant',
 'Punjabi Restaurant',
 'Mughlai Restaurant',
 'Modern European Restaurant',
 'English Restaurant',
 'Scandinavian Restaurant',
 'Eastern European Restaurant',
 'Parsi Restaurant',
 'Middle Eastern Restaurant',
 'Northeast Indian Restaurant',
 'Seafood Restaurant',
 'Indian Chinese Restaurant',
 'Falafel 

In [35]:
# Taking sum of all columns containing Restaurant and populate the result in Restaurants column
delhi_onehot['Restaurants']=delhi_onehot.loc[:,VenueCategory_with_Restaurants].sum(axis=1)
delhi_onehot

,Neighborhoods,ATM,Afghan Restaurant,Airport,Airport Food Court,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,...,Trail,Train Station,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Women's Store,Yoga Studio,Restaurants
0,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2547,West Patel Nagar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2548,West Patel Nagar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2549,West Patel Nagar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2550,West Patel Nagar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [36]:
delhi_grouped = delhi_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(delhi_grouped.shape)
delhi_grouped

(132, 199)


,Neighborhoods,ATM,Afghan Restaurant,Airport,Airport Food Court,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,...,Trail,Train Station,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Women's Store,Yoga Studio,Restaurants
0,Ashok Nagar (Delhi),0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.300000
1,Ashram Chowk,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.5,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.250000
2,Babarpur,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.5,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.250000
3,"Badarpur, Delhi",0.0,0.0,0.0,0.0,0.0,0.034483,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.379310
4,Bali Nagar,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,Vasant Kunj,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.022727,0.0,0.000000,0.250000
128,"Vasant Vihar, Delhi",0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.166667,0.166667
129,Vivek Vihar subdivision,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.076923
130,West Patel Nagar,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.437500


#### Create a new DataFrame for Restaurants data only

In [37]:
delhi_restaurant = delhi_grouped[["Neighborhoods","Restaurants"]]
delhi_restaurant

,Neighborhoods,Restaurants
0,Ashok Nagar (Delhi),0.300000
1,Ashram Chowk,0.250000
2,Babarpur,0.250000
3,"Badarpur, Delhi",0.379310
4,Bali Nagar,0.000000
...,...,...
127,Vasant Kunj,0.250000
128,"Vasant Vihar, Delhi",0.166667
129,Vivek Vihar subdivision,0.076923
130,West Patel Nagar,0.437500


## 7. Cluster Neighborhoods

##### Run k-means to cluster the Delhi areas into 3 clusters.

In [38]:
# set number of clusters
kclusters = 3

delhi_grouped_clustering = delhi_restaurant.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(delhi_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 2, 1, 0, 1, 2, 2, 2], dtype=int32)

In [39]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
delhi_merged = delhi_restaurant.copy()

# add clustering labels
delhi_merged["Cluster Labels"] = kmeans.labels_

delhi_merged.head()

,Neighborhoods,Restaurants,Cluster Labels
0,Ashok Nagar (Delhi),0.30000,0
1,Ashram Chowk,0.25000,0
2,Babarpur,0.25000,0
3,"Badarpur, Delhi",0.37931,2
4,Bali Nagar,0.00000,1


In [40]:
delhi_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
delhi_merged.head()

,Neighborhood,Restaurants,Cluster Labels
0,Ashok Nagar (Delhi),0.30000,0
1,Ashram Chowk,0.25000,0
2,Babarpur,0.25000,0
3,"Badarpur, Delhi",0.37931,2
4,Bali Nagar,0.00000,1


In [42]:
# merge delhi_merged with delhi_df to add latitude/longitude for each neighborhood
delhi_merged = delhi_merged.join(delhi_df.set_index("Neighborhood"), on="Neighborhood")

print(delhi_merged.shape)
delhi_merged.head()

ValueError: columns overlap but no suffix specified: Index(['Latitude', 'Longitude'], dtype='object')

In [43]:
# sort the results by Cluster Labels
print(delhi_merged.shape)
delhi_merged.sort_values(["Cluster Labels"], inplace=True)
delhi_merged 

(132, 5)


,Neighborhood,Restaurants,Cluster Labels,Latitude,Longitude
0,Ashok Nagar (Delhi),0.300000,0,28.690420,77.176060
86,Paharganj,0.200000,0,28.591080,77.091190
85,Old Delhi,0.270833,0,28.645960,77.214930
83,Nizamuddin West,0.166667,0,28.532490,77.278380
74,Nangloi Jat,0.200000,0,28.633650,77.136740
...,...,...,...,...,...
19,Delhi Cantonment,0.394737,2,28.594895,77.167263
17,Dayanand Colony,0.380000,2,28.573000,77.233600
16,Dashrath puri,0.326531,2,28.560560,77.246790
13,"Dabri, New Delhi",0.400000,2,28.654568,77.233419


#### Finally, let's visualize the resulting clusters

In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(delhi_merged['Latitude'], delhi_merged['Longitude'], delhi_merged['Neighborhood'], delhi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [46]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## 8.Examine Clusters

#### Cluster 1

In [49]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 0]


,Neighborhood,Restaurants,Cluster Labels,Latitude,Longitude
0,Ashok Nagar (Delhi),0.300000,0,28.690420,77.176060
86,Paharganj,0.200000,0,28.591080,77.091190
85,Old Delhi,0.270833,0,28.645960,77.214930
83,Nizamuddin West,0.166667,0,28.532490,77.278380
74,Nangloi Jat,0.200000,0,28.633650,77.136740
...,...,...,...,...,...
39,Jia Sarai,0.200000,0,28.689676,77.140972
2,Babarpur,0.250000,0,28.507370,77.303470
37,Janakpuri,0.294118,0,28.583370,77.247140
38,Jangpura,0.261905,0,28.546430,77.188960


In [50]:
print(delhi_merged.loc[delhi_merged['Cluster Labels'] == 0].shape)

(62, 5)


#### Cluster 2

In [51]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 1]

,Neighborhood,Restaurants,Cluster Labels,Latitude,Longitude
117,"Shanti Niketan, Delhi",0.000000,1,28.67006,77.25175
79,New Delhi,0.000000,1,28.57810,77.26999
15,Daryaganj,0.000000,1,28.60568,77.08222
108,Sarai Kale Khan,0.000000,1,28.55032,77.28343
107,Saket District Centre,0.000000,1,28.58486,77.26356
42,"Kamla Nagar, New Delhi",0.000000,1,28.52165,77.08646
112,Seemapuri subdivision,0.062500,1,28.65390,77.29641
18,Defence Colony,0.000000,1,28.59153,77.12942
118,Shastri Park,0.000000,1,28.66443,77.05745
91,Paschim Vihar,0.076923,1,28.64780,77.16447


In [52]:
print(delhi_merged.loc[delhi_merged['Cluster Labels'] == 1].shape)

(30, 5)


#### Cluster 3

In [80]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 2]

,Neighborhood,Restaurants,Cluster Labels,Latitude,Longitude
9,Chandni Chowk,0.480000,2,28.538410,77.248310
124,Tilak Nagar (Delhi),0.400000,2,28.665980,77.215200
114,Shahpur Jat,0.333333,2,28.670370,77.174140
8,Chanakyapuri,0.333333,2,28.656240,77.232330
109,Sarita Vihar,0.375000,2,28.575700,77.193620
125,Tis Hazari,0.387097,2,28.649890,77.235145
3,"Badarpur, Delhi",0.379310,2,28.652180,77.129775
7,Bhajanpura,0.615385,2,28.595060,77.185730
11,"Civil Lines, Delhi",0.380000,2,28.633920,77.219780
120,Shivaji Place,0.416667,2,28.562470,77.215270


In [53]:
print(delhi_merged.loc[delhi_merged['Cluster Labels'] == 2].shape)

(40, 5)


## 9.Observation:

Examining Clusters we can see cluster 1 has moderate number of restaurants in neighborhoods,cluster 2 also has low to moderate number of restaurants while cluster 3 has large number restaurants.Hence to start a profitable business of opening restaurants,cluster 2 is best and competition is less here compared to other clusters.Cluster 1 is also good place to start business where competition is greater than cluster 2 and less than cluster 3.
In brief most of the restaurants in Delhi are concentrated in Central area,where North-East part is best to set up restaurants for investors.